In [173]:
import pandas as pd
import json
from tqdm import tqdm
import numpy as np

In [174]:
# Loading data :
df = pd.read_csv('data/csv/int_matchs_results_1872_2021.csv',index_col = 0)
country_codes = pd.read_csv('data/csv/countries_codes.csv',dtype="str")

# Loading dictionaries where replacement and additional values are stored :
dictio_replace = json.load(open("data/json/dict_replace_country_code.json",'rb'))
dict_replace_foot_data = json.load(open("data/json/dict_replace_foot_data.json",'rb'))
uk_countries = json.load(open("data/json/uk_countries.json","rb"))

# Make index to datetime :
df.index = pd.to_datetime(df.index)

# Replace country names to make it consistent alltogether :
country_codes['name'] = country_codes['name'].replace(dictio_replace)
df['home_team'] = df['home_team'].replace(dict_replace_foot_data)
df['away_team'] = df['away_team'].replace(dict_replace_foot_data)

# Remove nans
df = df.dropna()

# Remove matches where same country (e.g. Germany vs Germany) because of replacement :
df = df.drop(df[df['away_team'] == df['home_team']].index)

# Keep only country code and name :
country_codes = country_codes[['name','country-code','region']]

# Add UK sub-countries to country_codes :
for ct in uk_countries:
    new = pd.DataFrame([[ct,uk_countries[ct],'Europe']],
    columns = ['name','country-code','region'])
    country_codes = country_codes.append(new,ignore_index=True)
country_codes.to_csv('data/csv/countries_codes_with_uk_subcountries.csv')

# Keep only countries that are in country codes list :
df = df[df['home_team'].isin(country_codes['name'].unique())]
df = df[df['away_team'].isin(country_codes['name'].unique())]

In [175]:
df.to_csv("data/csv/int_results.csv")

In [176]:
# Calculate number of scored goals per year :
year_index = df.index.to_period("Y")
yearly_home_scored_goals = df.groupby([year_index,'home_team']).sum().reset_index()[[
    'date',
    'home_team',
    'home_score'
    ]].rename(
        columns = {
            'home_team':'team',
            'home_score':'scored_goals'
        }
    )
yearly_away_scored_goals = df.groupby([year_index,'away_team']).sum().reset_index()[
    [
        'date',
        'away_team',
        'away_score'
    ]
].rename(
    columns = {
        'away_team':'team',
        'away_score':'scored_goals'
    }
)
yearly_total_scored_goals = pd.concat((
    yearly_home_scored_goals,
    yearly_away_scored_goals
    ),
    ignore_index = True)
yearly_total_scored_goals = yearly_total_scored_goals.groupby([
    'date',
    'team'
    ]).sum().reset_index()

yearly_total_scored_goals.to_csv('data/csv/yearly_scored_goals_by_team.csv',index=False)

In [177]:
# Calculate number of conceded goals per year :
yearly_home_conceded_goals = df.groupby([year_index,'home_team']).sum().reset_index()[[
    'date',
    'home_team',
    'away_score'
    ]].rename(
        columns = {
            'home_team':'team',
            'away_score':'conceded_goals'
        }
    )
yearly_away_conceded_goals = df.groupby([year_index,'away_team']).sum().reset_index()[
    [
        'date',
        'away_team',
        'home_score'
    ]
].rename(
    columns = {
        'away_team':'team',
        'home_score':'conceded_goals'
    }
)
yearly_total_conceded_goals = pd.concat((
    yearly_home_conceded_goals,
    yearly_away_conceded_goals
    ),
    ignore_index = True)
yearly_total_conceded_goals = yearly_total_conceded_goals.groupby([
    'date',
    'team'
    ]).sum().reset_index()

yearly_total_conceded_goals.to_csv('data/csv/yearly_conceded_goals_by_team.csv',index=False)

In [178]:
# Merge scored and conceded in a single dataframe
yearly_scored_conceded = yearly_total_scored_goals.merge(yearly_total_conceded_goals,on=['date','team'])
yearly_scored_conceded.to_csv("data/csv/yearly_scored_conceded_by_team.csv",index=False)

In [179]:
# Calculate number of wins, losses and draws by year :
home_wins_df = df.copy()
home_wins_df['wins'] = home_wins_df['home_score'] > home_wins_df['away_score']
home_wins_df['lost'] = home_wins_df['home_score'] < home_wins_df['away_score']
home_wins_df['draws'] = home_wins_df['home_score'] == home_wins_df['away_score']
home_wins_df = home_wins_df[
    [
        'home_team',
        'wins',
        'lost',
        'draws',
    ]
].rename(
    columns = {'home_team':'team'}
)
home_wins_df = home_wins_df.groupby([year_index,'team']).sum().reset_index()

away_wins_df = df.copy()
away_wins_df['wins'] = away_wins_df['home_score'] < away_wins_df['away_score']
away_wins_df['lost'] = away_wins_df['home_score'] > away_wins_df['away_score']
away_wins_df['draws'] = away_wins_df['home_score'] == away_wins_df['away_score']
away_wins_df = away_wins_df[
    [
        'away_team',
        'wins',
        'lost',
        'draws'
    ]
].rename(
    columns = {'away_team':'team'}
)
away_wins_df = away_wins_df.groupby([year_index,'team']).sum().reset_index()

yearly_total_wins = pd.concat((home_wins_df,away_wins_df),ignore_index=True).groupby(
    [
        'date',
        'team'
    ]
).sum().reset_index()
yearly_total_wins = yearly_total_wins.sort_values(by="date")
yearly_total_wins['date'] = yearly_total_wins['date'].astype('str').astype('int')
yearly_total_wins.to_csv('data/csv/yearly_wins_per_team.csv',index=False)

In [180]:
# years = list(range(1872,2022))

# teams = yearly_total_wins['team'].unique()
# cols = ["date","team","wins"]
# dfs = []
# for team in tqdm(teams):
#     team_df = yearly_total_wins[yearly_total_wins['team'] == team][cols]
#     team_years = list(team_df['date'].unique())
#     missing_years = [y for y in years if y not in team_years]
#     i = team_df.index[-1] + 1
#     cum_wins = team_df['wins'].cumsum()
#     team_df['wins'] = cum_wins
#     for y in missing_years:
#         team_df.loc[i] = [y,team,np.nan]
#         i+=1
#     team_df = team_df.sort_values(by="date")
#     team_df = team_df.fillna(method="ffill")
#     team_df = team_df.fillna(0)
#     dfs.append(team_df)
# yearly_cumulated_wins = pd.concat(dfs,ignore_index=True)
# yearly_cumulated_wins.to_csv('data/csv/yearly_cumulated_wins_per_team.csv',index=False)

In [181]:
# Compute country codes from Flagpedia :
cc_codes = json.load(open("data/json/cc_codes.json",'rb'))
codes_cc = {cc_codes[code]:code for code in cc_codes}
pd.DataFrame(codes_cc,index=[0]).to_csv("data/csv/flags_dataset_flagcdn.csv",index=False)

In [182]:
#Keep only selected tournaments :
df2 = df.copy()
df2['matches'] = 1

to_keep_tournaments = json.load(open("data/json/keep_tournaments.json",'rb'))
all_tournaments = list(df2['tournament'].unique())
dict_replace_tournaments = {
    t:'Other' for t in all_tournaments if t not in to_keep_tournaments
}

df2['tournament'] = df2['tournament'].replace(dict_replace_tournaments)

In [183]:
#Compute goal difference by tournament and by team :
home_scored_goals_by_tournament = df2.groupby(['home_team','tournament']).sum().reset_index()[[
    'tournament',
    'home_team',
    'home_score',
    'matches'
    ]].rename(
        columns = {
            'home_team':'team',
            'home_score':'scored_goals'
        }
    )
away_scored_goals_by_tournament = df2.groupby(['away_team','tournament']).sum().reset_index()[
    [
        'tournament',
        'away_team',
        'away_score',
        'matches'
    ]
].rename(
    columns = {
        'away_team':'team',
        'away_score':'scored_goals'
    }
)

scored_goals_by_tournament = pd.merge(
    home_scored_goals_by_tournament,
    away_scored_goals_by_tournament,
    on=['tournament','team'],
    how='outer'
    ).fillna(0)


scored_goals_by_tournament['scored_goals'] = (
    scored_goals_by_tournament['scored_goals_x']
    +
    scored_goals_by_tournament['scored_goals_y']
)
scored_goals_by_tournament['matches'] = (
    scored_goals_by_tournament['matches_x']
    +
    scored_goals_by_tournament['matches_y']
)
scored_goals_by_tournament = scored_goals_by_tournament.drop(columns = [
    'scored_goals_x',
    'scored_goals_y',
    'matches_x',
    'matches_y'
])

In [184]:
#Compute conceded goals by team and by tournament :
home_conceded_goals_by_tournament = df2.groupby(['home_team','tournament']).sum().reset_index()[[
    'tournament',
    'home_team',
    'away_score',
    'matches'
    ]].rename(
        columns = {
            'home_team':'team',
            'away_score':'conceded_goals'
        }
    )
away_conceded_goals_by_tournament = df2.groupby(['tournament','away_team']).sum().reset_index()[
    [
        'tournament',
        'away_team',
        'home_score',
        'matches'
    ]
].rename(
    columns = {
        'away_team':'team',
        'home_score':'conceded_goals'
    }
)

conceded_goals_by_tournament = pd.merge(
    home_conceded_goals_by_tournament,
    away_conceded_goals_by_tournament,
    on=['tournament','team'],
    how='outer'
    ).fillna(0)


conceded_goals_by_tournament['conceded_goals'] = (
    conceded_goals_by_tournament['conceded_goals_x']
    +
    conceded_goals_by_tournament['conceded_goals_y']
)
conceded_goals_by_tournament['matches'] = (
    conceded_goals_by_tournament['matches_x']
    +
    conceded_goals_by_tournament['matches_y']
)
conceded_goals_by_tournament = conceded_goals_by_tournament.drop(columns = [
    'conceded_goals_x',
    'conceded_goals_y',
    'matches_x',
    'matches_y'
])

In [185]:
goals_by_tournament = pd.merge(
    scored_goals_by_tournament,
    conceded_goals_by_tournament.drop(columns = [
        'matches'
    ]),
    on=[
        'team',
        'tournament'
        ]
        )

In [186]:
goals_by_tournament['goal_difference'] = (
    goals_by_tournament['scored_goals']
    - 
    goals_by_tournament['conceded_goals']
)
goals_by_tournament['goal_difference_average'] = (
    goals_by_tournament['goal_difference'] 
    /
    goals_by_tournament['matches']
)
goals_by_tournament = goals_by_tournament.drop(
    columns = [
        'scored_goals',
        'conceded_goals'
    ]
)

In [187]:
# Calculate number of wins, losses and draws by tournament and by team :
home_wins_tournament = df2.copy()
home_wins_tournament['wins'] = home_wins_tournament['home_score'] > home_wins_tournament['away_score']
home_wins_tournament['lost'] = home_wins_tournament['home_score'] < home_wins_tournament['away_score']
home_wins_tournament['draws'] = home_wins_tournament['home_score'] == home_wins_tournament['away_score']
home_wins_tournament = home_wins_tournament[
    [
        'home_team',
        'wins',
        'lost',
        'draws',
        'tournament',
        'matches'
    ]
].rename(
    columns = {'home_team':'team'}
)
home_wins_tournament = home_wins_tournament.groupby(['tournament','team']).sum().reset_index()

away_wins_tournament = df2.copy()
away_wins_tournament['wins'] = away_wins_tournament['home_score'] < away_wins_tournament['away_score']
away_wins_tournament['lost'] = away_wins_tournament['home_score'] > away_wins_tournament['away_score']
away_wins_tournament['draws'] = away_wins_tournament['home_score'] == away_wins_tournament['away_score']
away_wins_tournament = away_wins_tournament[
    [
        'away_team',
        'wins',
        'lost',
        'draws',
        'tournament',
        'matches'
    ]
].rename(
    columns = {'away_team':'team'}
)
away_wins_tournament = away_wins_tournament.groupby(["tournament",'team']).sum().reset_index()

tournament_total_wins = pd.merge(home_wins_tournament,away_wins_tournament,on = [
    'tournament',
    "team"
],how='outer')

tournament_total_wins = tournament_total_wins.fillna(0)

tournament_total_wins["wins"] = (
    tournament_total_wins['wins_x']
    +
    tournament_total_wins['wins_y']
)
tournament_total_wins = tournament_total_wins[
    [
        'tournament',
        'team',
        'wins'
    ]
]

In [188]:
goal_difference_wins_tournament = pd.merge(tournament_total_wins,goals_by_tournament,
on=['tournament','team'],how='outer')

In [189]:
goal_difference_wins_tournament['win_ratio'] = (
    goal_difference_wins_tournament['wins']
    /
    goal_difference_wins_tournament['matches']
)

In [190]:
goal_difference_wins_tournament['continent'] = goal_difference_wins_tournament['team'].map(
    country_codes.set_index('name')['region'].to_dict()
)
goal_difference_wins_tournament['continent'] = goal_difference_wins_tournament['continent'].replace(
    {"Americas":"America"}
)

In [191]:
goal_difference_wins_tournament.to_csv("data/csv/goal_difference_by_tournament_and_team.csv",index=False)

In [192]:
all_tournaments_wins_gd = goal_difference_wins_tournament.groupby('team').sum()[
    [
    "wins",
    "matches",
    "goal_difference"]
].reset_index()

all_tournaments_wins_gd["win_ratio"] = (
    all_tournaments_wins_gd["wins"] 
    /
    all_tournaments_wins_gd["matches"]
)

all_tournaments_wins_gd["goal_difference_average"] = (
    all_tournaments_wins_gd["goal_difference"] 
    /
    all_tournaments_wins_gd["matches"]
)

In [193]:
all_tournaments_wins_gd['continent'] = all_tournaments_wins_gd['team'].map(
    country_codes.set_index('name')['region'].to_dict()
)
all_tournaments_wins_gd['continent'] = all_tournaments_wins_gd['continent'].replace(
    {"Americas":"America"}
)

In [194]:
all_tournaments_wins_gd.to_csv("data/csv/all_tournaments_goal_difference.csv",index=False)